In [1]:
# base
import re, pickle
import numpy as np

# tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
with open('data/trump_raw_text.txt', 'r', encoding='utf8') as myfile:
    raw_text = myfile.read()

In [3]:
def clean_text(t):
    # to lower
    t = t.lower()
    # remove quotes
    t = re.sub(r'"@.*', '', t)
    t = re.sub(r'^“.*”$', '', t)
    # remove URLs
    t = re.sub(r'https*:\/\/\S*', '', t)
    t = re.sub(r'pic\.twitter\.com\/\S*', '', t)
    # remove \n
    t = re.sub('\n', ' ', t)
    # remove extra whitespaces
    t = re.sub(r'\s+', ' ', t)
    # replace '&amp' with 'and'
    t = re.sub('&amp;', 'and', t)     
    # replace abbreviations
    t = re.sub("'ll", ' will', t)
    t = re.sub("won't", 'will not', t)
    t = re.sub("n't", ' not', t) 
    # remove @mention
    t = re.sub(r'@[A-Za-z0-9_]+', '', t) 
    # remove #tag
    t = re.sub(r'#[A-Za-z0-9_]+', '', t) 
    # remove special characters
    t = re.sub(r'[^a-zA-Z ]', '', t) 
    # remove multiple spaces 
    t = re.sub("\s\s+", " ", t) 
    return t

raw_text = clean_text(raw_text)

In [4]:
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  920369
Total Vocab:  27


In [5]:
with open('data/chars.txt', 'wb') as fp:
    pickle.dump(chars, fp)

In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.1, random_state=42)

print("Total Patterns: ", len(X_train))

Total Patterns:  828242


In [12]:
# reshape X to be [samples, time steps, features]
X = np.reshape(X_train, (len(X_train), seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(y_train)
# define the LSTM model
model = Sequential([
    LSTM(256, dropout=0.2, input_shape=(None, X.shape[2]), return_sequences=True),
    LSTM(256, dropout=0.2, return_sequences=True),
    LSTM(256, dropout=0.2),
    Dense(y.shape[1], activation='softmax')
])

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [13]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model_save = ModelCheckpoint('models/trump_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# fit the model
model.fit(X, y, epochs=100, batch_size=128, callbacks=[early_stop, model_save], validation_split=0.1)

Train on 745417 samples, validate on 82825 samples
Epoch 1/100
745417/745417 [==============================] - 324s 435us/sample - loss: 2.2730 - val_loss: 1.8159
Epoch 2/100
745417/745417 [==============================] - 317s 425us/sample - loss: 1.8135 - val_loss: 1.6243
Epoch 3/100
745417/745417 [==============================] - 312s 419us/sample - loss: 1.6656 - val_loss: 1.5120
Epoch 4/100
745417/745417 [==============================] - 293s 393us/sample - loss: 1.5815 - val_loss: 1.4399
Epoch 5/100
745417/745417 [==============================] - 328s 440us/sample - loss: 1.5252 - val_loss: 1.5414
Epoch 6/100
745417/745417 [==============================] - 327s 439us/sample - loss: 1.4835 - val_loss: 1.3894
Epoch 7/100
745417/745417 [==============================] - 328s 440us/sample - loss: 1.4501 - val_loss: 1.3439
Epoch 8/100
745417/745417 [==============================] - 315s 423us/sample - loss: 1.4247 - val_loss: 1.3331
Epoch 9/100
745417/745417 [==================